In [1]:
import matplotlib.pyplot as plt
import matplotlib

%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2


import numpy as np
#%matplotlib inline

import os
from utils import *

import os

from utils_calcium import ProcessCalcium

Autosaving every 180 seconds


In [2]:
####################################################
################## LOAD DATA #######################
####################################################

root_dir = '/media/cat/4TB/donato/andres/master_analysis/'

# ****************** MICE AVAILABLE ***********************
# M1 = [
#     'DON-011733',
#     'DON-014382',
#     'DON-014451',
#     'DON-014575',
#     'DON-014618',
#     'DON-015821',

# CA3 = [
#     'DON-012502',
#     'DON-014266',
#     'DON-014371',
#     'DON-015801',
# ]
# *********************************************************

#
animal_id = 'DON-014371'

#       
c = ProcessCalcium(root_dir,
                   animal_id)

#
c.load_data()

#
c.load_day0_mask()



  0%|          | 0/9 [00:00<?, ?it/s]

loading binarization


 11%|█         | 1/9 [00:01<00:14,  1.80s/it]

loading binarization


 33%|███▎      | 3/9 [00:05<00:10,  1.72s/it]

...

loading binarization
loading binarization


 78%|███████▊  | 7/9 [00:11<00:03,  1.61s/it]

loading binarization


100%|██████████| 9/9 [00:14<00:00,  1.65s/it]

loading binarization


In [15]:
######################################################
######################################################
######################################################

def convert_ca_to_spike_time_format(rasters):

    print (rasters.shape)

    # get the locations of the spikes as 1s in the raster
    spike_locs = np.where(rasters == 1)

    # make a 2 dimensional array with spike times in the first column and neuron number in the second column
    spike_times = np.zeros((spike_locs[0].shape[0], 2), dtype = np.int32)
    spike_times[:,0] = spike_locs[1]
    spike_times[:,1] = spike_locs[0]
    
    # sort the array by spike time which is the first column
    idx = spike_times[:,0].argsort()
    spike_times = spike_times[idx]

    #
    print (spike_times.shape)
    print (spike_times)

    #
    return spike_times

#
rasters = c.sessions[1].F_upphase_bin

#
spike_times = convert_ca_to_spike_time_format(rasters)

# find trials as locations 
# save the times_snipets_over_threshold to disk
np.save(os.path.join(c.sessions[1].data_dir,
        'spike_times.npy'), spike_times)




(318, 90000)
(435793, 2)
[[    3   245]
 [    3    37]
 [    3    26]
 ...
 [89993    69]
 [89994    69]
 [89995    69]]


In [10]:
#############################################################
####### COMPUTE TONE TRIGGERED TIME-FIELDS - ALL CELLS ######
#############################################################

#
F_type = 'upphase'
session_id = 1
c.session_id = session_id

# compute the tone triggered 
c.window = 5*30  # window to visualize in frames
c.vmax = 0.3
c.vmin = None
c.shuffle = False
c.F_type = F_type   # 'upphase' or 'F_filtered'
c.smooth = False
c.remove_rewards = True
c.recompute_tones = False

c.recompute_tones = True

#
c.tone_type = 'starts'  # 'starts' vs 'all'

#
c.cmap = 'jet'
c.save_to_disk = True

#
c.get_tones()

#
c.get_unique_tones()

#
#c.get_tone_triggered_ca_all_cells()


In [38]:
########################################################
########################################################
########################################################
 
# load data from spreassheetds
df = pd.read_excel('/media/cat/4TB/donato/andres/master_analysis/DON-014371/20230331/results.xlsx')

# get threshold: 3rd column of df as a numpy array 
thresh = df.iloc[:, 2].values

# get ensemble state: 8th column of df as a numpy array
e_state = df.iloc[:, 7].values

# delete first few frames
e_state[:100] = 0 

# filter using savgol_filter
from scipy.signal import savgol_filter
e_state = savgol_filter(e_state, 31, 3)



In [127]:
#######################################
#######################################
#######################################

def plot_e_state_binarization(e_state,
                              e_state_bin,
                              e_state_sign,
                              thresh):
    #
    t = np.arange(0, 90000, 1)/30.

    # plot ensemble state graph
    plt.figure(figsize=(10, 10))

    plt.plot(t,e_state, label = 'ensemble state')
    plt.plot(t,e_state_bin*e_state_sign, label = 'ensemble state bin')

    # plot horizontal line from 0 to 90000 at thresh
    plt.plot(t, thresh, 'k-', lw=2, label = 'thresh')

    #
    plt.xlabel('time (s)')

    # set xlims
    plt.xlim(t[0], t[-1])

    plt.legend()
    plt.show()


#
c.sessions[1].percentile_threshold = 0.98

#
c.sessions[1].binarize_traces_single_trace(np.abs(e_state))

# get locations of positive and negative values of e_state
pos = np.where(e_state >= 0)[0]
neg = np.where(e_state < 0)[0]

# make an array of -1 and +1 to represent the sign of the e_state
e_state_sign = np.zeros(e_state.shape)
e_state_sign[pos] = 1
e_state_sign[neg] = -1

#
e_state_bin = c.sessions[1].trace_upphase_bin

#
plot_e_state_binarization(e_state,
                        e_state_bin,
                        e_state_sign,
                       thresh)


model filter: remove bleaching or trends: 100%|██████████| 1/1 [00:00<00:00, 351.34it/s]


binarizing continuous traces filtered fluorescence onphase: 100%|██████████| 1/1 [00:00<00:00, 100.07it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|██████████| 1/1 [00:00<00:00, 91.61it/s]


In [128]:
#######################################
#######################################
#######################################

def process_snippets(c):
    # show extracted snippets
    starts_ends = np.array(c.sessions[1].starts_ends).squeeze()
    print (starts_ends.shape)

    # get the sign of each extracted snippet
    signs = []
    for k in range(starts_ends.shape[0]):
        signs.append(np.median(e_state_sign[starts_ends[k,0]:starts_ends[k,1]]))

    signs = np.int32(signs)

    # also check which positive snippets cross the threshold

    # 
    rewarded_snippets = []
    for k in range(starts_ends.shape[0]):

        if signs[k] == 1:
            temp = e_state[starts_ends[k,0]:starts_ends[k,1]]
            if np.max(temp) > np.mean(thresh):
                if np.min(temp) < np.mean(thresh)*.5:

                    rewarded_snippets.append(k)
                

    return signs, np.array(rewarded_snippets)

#
c.sessions[1].thresh = thresh
signs, rewarded_snippets = process_snippets(c)




(114, 2)


In [ ]:
###############################################
###############################################
###############################################
#
def time_warp_snippet(snippet):

    #
    time_original = np.arange(snippet.shape[0])
    data_original = snippet

    #
    f = scipy.interpolate.interp1d(time_original, data_original, kind='linear')

    # Use this function to obtain new data samples
    time_new = np.linspace(0, time_original[-1], 10000)
    #idx = np.where(time_new < time_original[-1])[0]
    #time_new = time_new[idx]

    data_new = f(time_new)
    #print (data_new.shape)
    
    # take a total of 100 samples from the snippet
    idx = np.linspace(0, data_new.shape[0]-1, 30).astype(int)
    snippet_downsampled = data_new[idx]

    return snippet_downsampled

#
time_warp = True


# plot the ensemble state for each of the starts_ends separating into 2 plots by sign
plt.figure(figsize=(15,5))

#
ax1 = plt.subplot(1,3,1)
ax1.set_title('positive')
plt.xlabel('time (s)')
# plot the thresh value
ax1.axhline(y=np.mean(thresh), color='grey', linestyle='--')

#
ax2 = plt.subplot(1,3,2)
ax2.set_title('negative')
plt.xlabel('time (s)')


# count # of positive and negative vals in signs
pos = np.sum(signs == 1)
neg = np.sum(signs == -1)

# make a colormap of fixed colors in viridis for 10 values
cmap_pos = plt.cm.viridis(np.linspace(0, 1, pos))
cmap_neg = plt.cm.viridis(np.linspace(0, 1, neg))

#
ctr_pos = 0
ctr_neg = 0
for k in range(starts_ends.shape[0]):
    temp = e_state[starts_ends[k,0]:starts_ends[k,1]]

    # if time warp
    if time_warp == True:
        temp = time_warp_snippet(temp)

    if signs[k] == 1:
        ax1.plot(np.arange(temp.shape[0])/30, temp,
                 c=cmap_pos[ctr_pos])
        ctr_pos += 1
    else:
        ax2.plot(np.arange(temp.shape[0])/30, temp,
                 c=cmap_neg[ctr_neg])
        ctr_neg += 1

#
ax3 = plt.subplot(1,3,3)
ax3.set_title('good snippets (> threshold; start from 0.5 max)')
plt.xlabel('time (s)')

#
cmap_r = plt.cm.viridis(np.linspace(0, 1, rewarded_snippets.shape[0]))
for k in range(rewarded_snippets.shape[0]):
    idx = rewarded_snippets[k]
    temp = e_state[starts_ends[idx,0]:starts_ends[idx,1]]
    # if time warp
    if time_warp == True:
        temp = time_warp_snippet(temp)

    ax3.plot(np.arange(temp.shape[0])/30, temp,
             c=cmap_r[k])

# plot the thresh value
ax3.axhline(y=np.mean(thresh), color='grey', linestyle='--')

#
plt.show()

In [27]:
##############################################
####### COMPUTE [ca] TRIGGERED TONES #########
##############################################

#
F_type = 'upphase'
session_ids = np.arange(1, len(c.session_ids))
session_id = 1

#
c.session_id = session_id

# compute the tone triggered 
c.window = 5*30  # window to visualize in frames
c.vmax = 50
c.vmin = None
c.shuffle = False
c.F_type = F_type   # 'upphase' or 'F_filtered'
c.smooth = False
c.remove_rewards = True
c.recompute_tones = False
c.remove_base_tone = True

#
c.tone_type = 'starts'  # 'starts' vs 'all'

#
c.cmap = 'jet'
c.save_to_disk = True

#
c.get_tones()

#
c.get_unique_tones()

#
c.get_ca_triggered_tone_all_cells()


  0%|          | 0/318 [00:00<?, ?it/s]

In [41]:
######################################################
########### COMPUTE TONE TRIGGERED STATIONARY VALS ###
######################################################

c.session_id = 1
c.get_tones()

c.get_unique_tones()

# compute the tone triggered 

#c.min_ca_val = 0.15
c.window = 15*30  # window to visualize in frames
c.vmax = 0.3
c.vmin = None
c.shuffle = False
c.F_type = F_type   # 'upphase' or 'F_filtered'
c.smooth = True
c.remove_rewards = True

c.cmap = 'jet'
c.save_to_disk = True
plt.figure()
for cell_id in trange(c.sessions[c.session_id].F_filtered.shape[0]):
    c.cell_id = cell_id
    c.get_tone_triggered_ca_single_cell_static()


100%|██████████| 318/318 [00:17<00:00, 18.06it/s]


In [ ]:
####################################################
####################################################
####################################################

# compute the tone triggered 
c.min_ca_val = 0.15
c.window = 15*30
c.vmax = 0.2
c.vmin = -c.vmax
c.cmap = 'viridis'

c.cell_id = 0
c.save_to_disk = True
c.get_tone_triggered_ca_single_cell()

#

In [ ]:
####################################################
####### PLOT MASK VS. CLOSESST SUITE2P CELLS #######
####################################################
# loop through contours rois list and plot them

#
c.contour_ROI_max_dist = 20

# gets and plots the most nearby cells to each contour
c.plot_matching_contours()

#


In [ ]:
######################################################
####### LOAD PERI-REWARD HISTOGRAM FOR ALL CELLS #####
######################################################

#
c.reward_window = 450
c.shuffle_rewards = False
c.filter = True
c.compute_reward_centered_traces()


  0%|          | 0/9 [00:00<?, ?it/s]

# of cells, times:  (627, 90000) , output:  (39, 627, 900)
# of cells, times:  (623, 90000) , output:  (35, 623, 900)
# of cells, times:  (699, 90000) , output:  (14, 699, 900)
# of cells, times:  (681, 90000) , output:  (41, 681, 900)
# of cells, times:  (723, 90000) , output:  (9, 723, 900)
# of cells, times:  (674, 90000) , output:  (58, 674, 900)
# of cells, times:  (761, 90000) , output:  (26, 761, 900)
# of cells, times:  (1130, 90000) , output:  (21, 1130, 900)


In [ ]:
######################################################
####### LOAD PERI-REWARD HISTOGRAM FOR ALL CELLS #####
######################################################

#
c.session_id = 8
c.scale = .5
c.shuffled = False
c.sort_by_peak = True
c.min_ptp = 0.15

c.plot_reward_centered_traces()


input traces:  (623, 900)
closest_ROI_cells:  [[  0  36]
 [  0  37]
 [  0  55]
 [  1 175]
 [  2 163]
 [  2 328]]
loopin gover all suite2p cells:  623
k:  36  matching_cell:  36  matching_mask:  0
k:  37  matching_cell:  37  matching_mask:  0
k:  55  matching_cell:  55  matching_mask:  0
k:  163  matching_cell:  163  matching_mask:  2
k:  175  matching_cell:  175  matching_mask:  1
k:  328  matching_cell:  328  matching_mask:  2
k:  193  clrs2[k][0]:  lightblue
k:  277  clrs2[k][0]:  blue
k:  278  clrs2[k][0]:  blue
k:  471  clrs2[k][0]:  blue


In [ ]:
# ######################################################
# ####### LOAD PERI-REWARD HISTOGRAM FOR ALL CELLS #####
# ######################################################

# # same but rank the cells by their peak response
# c.plot_reward_centered_traces_ranked()

In [ ]:
######################################################
####### LOAD PERI-REWARD HISTOGRAM FOR ALL CELLS #####
######################################################

#
c.plot_reward_centered_traces_ROIs_only()


closest_ROI_cells:  [[  0 142]
 [  0 180]
 [  0 258]
 [  1  23]
 [  2  40]
 [  2 143]
 [  3  27]
 [  3 107]
 [  3 138]
 [  3 194]
 [  3 245]]


Text(0.5, 0, 'Time (s)')

In [ ]:
# ######################################################
# ####### LOAD PERI-REWARD HISTOGRAM FOR ALL CELLS #####
# ######################################################

# # same but rank the cells by their peak response
# c.plot_reward_centered_traces_ROIs_only_ranked()


In [ ]:
######################################################
############### SHOW ALL DAYS/SESSIONS ###############
######################################################

#
c.reward_window = 450

#
c.plot_reward_centered_traces_ROIs_all_days()
plt.xlabel('Time (s)',fontsize=20)


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Text(0.5, 0, 'Time (s)')

{'suite2p_version': '0.10.1', 'look_one_level_down': 0.0, 'fast_disk': [], 'delete_bin': True, 'mesoscan': False, 'bruker': False, 'bruker_bidirectional': False, 'h5py': [], 'h5py_key': 'data', 'save_path0': [], 'save_folder': [], 'subfolders': [], 'move_bin': False, 'nplanes': 1, 'nchannels': 1, 'functional_chan': 1, 'tau': 1.0, 'fs': 30.947, 'force_sktiff': False, 'frames_include': -1, 'multiplane_parallel': 0.0, 'ignore_flyback': [], 'preclassify': 0.0, 'save_mat': True, 'save_NWB': 0.0, 'combined': 1.0, 'aspect': 1.0, 'do_bidiphase': False, 'bidiphase': 0.0, 'bidi_corrected': False, 'do_registration': 1, 'two_step_registration': 0.0, 'keep_movie_raw': False, 'nimg_init': 500, 'batch_size': 500, 'maxregshift': 0.1, 'align_by_chan': 1, 'reg_tif': True, 'reg_tif_chan2': False, 'subpixel': 10, 'smooth_sigma_time': 0.0, 'smooth_sigma': 1.15, 'th_badframes': 100.0, 'norm_frames': True, 'force_refImg': False, 'pad_fft': False, 'nonrigid': True, 'block_size': [128, 128], 'snr_thresh': 1.2,

In [ ]:
# load yaml file
import yaml
yaml_file = '/media/cat/4TB/donato/andres/master_analysis/DON-014266/DON-014266.yaml'
with open(yaml_file) as file:
    #
    data = yaml.load(file, Loader=yaml.FullLoader)

    print (data['session_names'])

['day0', 20230131, 20230201, 20230202, 20230203, 20230204, 20230205, 20230206, 20230207]


In [ ]:
import numpy as np

#
d = np.load('/media/cat/4TB/donato/tests/test1/data/results.npz',allow_pickle=True)
ttl_voltages1 = d['ttl_voltages']
ttl_times1 = d['ttl_times']

#
d2 = np.load('/media/cat/4TB/donato/tests/test2/data/results.npz',allow_pickle=True)
ttl_voltages2 = d2['ttl_voltages']
ttl_times2 = d2['ttl_times']

print (ttl_voltages1.shape)
print (ttl_voltages2.shape)

#
print (ttl_voltages1)
print (ttl_voltages2)

# plot ttl_voltages
plt.figure()
ax=plt.subplot(211)
plt.plot(ttl_voltages1, label='298um',
         c='blue')
plt.xlim(0,70000)

#

plt.legend()
plt.ylabel("Voltage from TTL pulse")
ax=plt.subplot(212)
plt.plot(ttl_voltages2,label='647um',
         c='red')
plt.xlim(0,70000)


#

plt.legend()
plt.ylabel("Voltage from TTL pulse")


plt.show()


#
print (ttl_times1.shape)
print (ttl_times2.shape)


(65019, 1)
(38867, 1)
[[0.03918894]
 [0.03951282]
 [0.03935088]
 ...
 [0.03886507]
 [0.03886507]
 [0.03870313]]
[[0.03918894]
 [0.03918894]
 [0.03918894]
 ...
 [5.0910172 ]
 [5.06802013]
 [0.03935088]]
(758,)
(1000,)


In [ ]:
######################################################
####### PLOT PERI-REWARD HISTOGRAM FOR ALL CELLS #####
######################################################



# plot the graph
plt.figure()
for k in range(1,9,1):
    c.ax=plt.subplot(2,4,k)
    c.session_id = k
    c.reward_window = 180
    c.get_reward_centered_traces()

    #
    c.plot_network_graph()

#
plt.show()


